In [2]:
import requests
from requests.auth import HTTPBasicAuth


params = {
    'orderBy':'ingestionDate desc', 
    'platformname': 'Sentinel-2',
    'ingestiondate': '[NOW-1DAY]'
    
}

response = requests.get(
    'https://scihub.copernicus.eu/dhus/search?q=*&orderby=ingestiondate desc&platformname=Sentinel-2&ingestiondate=[NOW-1DAY]', 
    params=params,
    auth=HTTPBasicAuth('aadidevsooknanan', 'K5D5#4c3')
)

In [55]:
from snappy import ProductIO

ModuleNotFoundError: No module named 'snappy'

In [18]:
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [19]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [42]:
from datetime import datetime, timedelta

current_day = datetime.today().strftime('%Y-%m-%d')
previous_day = (datetime.today() - timedelta(days=15)).strftime('%Y-%m-%d')

print(previous_day, current_day)

2021-08-15 2021-08-30


In [43]:
images = ee.ImageCollection("MODIS/006/MOD14A1")\
            .filter(ee.Filter.date(previous_day, current_day))

In [47]:
fireMaskVis

In [44]:
fireMaskVis = {
  'min': 0.0,
  'max': 6000.0,
  'bands': ['MaxFRP', 'FireMask', 'FireMask'],
};


my_map = folium.Map(location=[20, 0], zoom_start=3, height=500)

# Add custom basemaps
# basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)
my_map.add_ee_layer(images.mean(), fireMaskVis, 'FireMask')
display(my_map)